https://towardsdatascience.com/17-rules-of-thumb-for-building-a-neural-network-93356f9930af

Following are the thumb-rules for building an MLP. However, most of them are applicable on other Deep Learning models.

Number of Layers: Start with two hidden layers (this does not include the last layer).

Number of nodes (size) of intermediate layers: a number from the geometric progression of 2, e.g., 4, 8, 16, 32, … . The first layer should be around half of the number of input data features. The next layer size as half of the previous.

Number of nodes (size) of output layer for Classification: If binary classification then the size is one. For a multi-class classifier, the size is the number of classes.
Size of output layer for regression: If single response then the size one. For multi-response regression, the size is the number of responses.

Activation for intermediate layers: Use relu activation.

Activation for output layer: Use sigmoid for binary classification, softmax for multi-class classifier, and linear for regression. For Autoencoders, the last layer should be linear if the input data is continuous, otherwise, sigmoid or softmax for binary or multi-level categorical input.

Dropout layers: Add Dropout after every layer, except the Input layer (if defining the Input layer separately). Set Dropout rate to 0.5. Dropout rate > 0.5 is counter-productive. If you believe a rate of 0.5 is regularizing too many nodes, then increase the size of the layer instead of reducing the Dropout rate to less than 0.5. I prefer to not set any Dropout on the Input layer. But if you feel compelled to do that, set the Dropout rate < 0.2.

Data preprocessing: I am assuming your predictors X is numeric and you have already converted any categorical columns into one-hot-encoding. Before using the data for model training, perform data scaling. UseMinMaxScaler from sklearn.preprocessing. If this does not work well, do StandardScaler present in the same library. The scaling is needed for y in regression.

Split data to train, valid, test: Use train_test_split from sklearn.model_selection. See example below.

Class weights: If you have unbalanced data, then set class weights to balance the loss in your model.fit . For a binary classifier, the weights should be: {0: number of 1s / data size, 1: number of 0s / data size}. For extremely unbalanced data (rare events), class weight may not work. Be cautious adding it.

Optimizer: Use adam with its default learning rate.

Loss in classification: For binary classification use binary_crossentropy. For multiclass, use categorical_crossentropy if the labels are one-hot-encoded, otherwise use sparse_categorical_crossentropy if the labels are integers.

Epochs: Start with 20 to see if the model training shows decreasing loss and any improvement in accuracy. If there is no minimal success with 20 epochs, move on. If you get some minimal success, make epoch as 100.

Batch size: Choose the batch size from the geometric progression of 2. For unbalanced datasets have larger value, like 128, otherwise start with 16.


tried the solutions here but none of them work

https://stackoverflow.com/questions/58479556/notimplementederror-cannot-convert-a-symbolic-tensor-2nd-target0-to-a-numpy

https://stackoverflow.com/questions/66775948/downgrade-python-version-from-3-7-to-3-6-in-google-colab

it is still an open issue on tensorflow github

https://github.com/tensorflow/models/issues/9706

In [1]:
# ! pip install -U tensorflow numpy --quiet

In [2]:
# %%bash

# MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
# MINICONDA_PREFIX=/usr/local
# wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
# chmod +x $MINICONDA_INSTALLER_SCRIPT
# ./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [3]:
# import sys
# _ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))

In [4]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tensorflow.keras import models
from tensorflow.keras import layers

from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix

In [5]:
DIR_PATH = "/content/drive/MyDrive/Colab Notebooks/ML CCC"

RANDOM_STATE = 42

In [6]:
f2_score = lambda y_test, y_pred: fbeta_score(y_test, y_pred, beta=2)
f2_scorer = make_scorer(fbeta_score, beta=2)

In [7]:
X_train = pd.read_csv(os.path.join(DIR_PATH, 'X_train_prep.csv')) 
X_test = pd.read_csv(os.path.join(DIR_PATH, 'X_test_prep.csv'))
y_train = pd.read_csv(os.path.join(DIR_PATH, 'y_train_prep.csv'))
y_test = pd.read_csv(os.path.join(DIR_PATH, 'y_test_prep.csv'))

In [8]:
y_train = y_train.to_numpy().ravel()
y_test = y_test.to_numpy().ravel()

In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14324 entries, 0 to 14323
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Gender              14324 non-null  int64  
 1   Own_car             14324 non-null  int64  
 2   Own_property        14324 non-null  int64  
 3   Work_phone          14324 non-null  int64  
 4   Phone               14324 non-null  int64  
 5   Email               14324 non-null  int64  
 6   Unemployed          14324 non-null  int64  
 7   Num_children        14324 non-null  float64
 8   Num_family          14324 non-null  float64
 9   Account_length      14324 non-null  float64
 10  Total_income        14324 non-null  float64
 11  Age                 14324 non-null  float64
 12  Years_employed      14324 non-null  float64
 13  Income_type_1       14324 non-null  int64  
 14  Income_type_2       14324 non-null  int64  
 15  Income_type_3       14324 non-null  int64  
 16  Inco

48/2 = 24
choose (32, 16)

> NotImplementedError: Cannot convert a symbolic tf.Tensor (strided_slice_1:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.
  

In [10]:
model = models.Sequential()
model.add(layers.Dense(
    49, input_shape=(49,), activation='relu'
))
model.add(layers.Dense(
    32, activation='relu'
))
model.add(layers.Dense(
    16, activation='relu'
))
model.add(layers.Dense(
    1, activation='sigmoid'
))
model.compile(
    loss='binary_crossentropy', optimizer='adam', metrics=[f2_score]
)
model.fit(X_train, y_train, epochs=20, batch_size=50)

Epoch 1/20


NotImplementedError: ignored

In [11]:
! python --version

Python 3.7.13


In [19]:
pip show pandas

Name: pandas
Version: 1.3.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, pytz, python-dateutil
Required-by: xarray, vega-datasets, statsmodels, sklearn-pandas, seaborn, pymc3, plotnine, pandas-profiling, pandas-gbq, pandas-datareader, mlxtend, mizani, holoviews, gspread-dataframe, google-colab, fix-yahoo-finance, fbprophet, fastai, cufflinks, cmdstanpy, arviz, altair
